In [8]:
import requests
import numpy as np
import pandas as pd
import datetime


In [9]:
'''
Vamos a utilizar la Api de chuck norris para obtener chistes aleatorios.

https://api.chucknorris.io/

Porque... why not?

'''

'\nVamos a utilizar la Api de chuck norris para obtener chistes aleatorios.\n\nhttps://api.chucknorris.io/\n\nPorque... why not?\n\n'

In [10]:
#Definimos la funcion para llamar la api y obtener un chiste aleatorio

def get_joke():
    url = 'https://api.chucknorris.io/jokes/random'
    response = requests.get(url)
    data = response.json()
    data = pd.DataFrame([data])

    data['fecha_insercion'] = datetime.datetime.now()
    data = data[['id', 'created_at', 'url', 'value', 'fecha_insercion']]
    return data

#probamos la funcion
joke = get_joke()
joke

,id,created_at,url,value,fecha_insercion
0,yNualoRcRYukITzU-yleow,2020-01-05 13:42:23.240175,https://api.chucknorris.io/jokes/yNualoRcRYukI...,Chuck Norris makes great charcoal grilled cann...,2024-05-16 21:56:30.141513


In [11]:
import config
user = config.user
passw = config.passw
host = config.host
port = config.port
database = config.database
schema = config.schema

In [12]:
#coneccion a redshift
import sqlalchemy

def conexion_redshift(user, passw, host, port, database):
    conn_string = f'postgresql://{user}:{passw}@{host}:{port}/{database}?sslmode=require'
    engine = sqlalchemy.create_engine(conn_string)
    conn = engine.connect()
    return conn, engine

conn, engine = conexion_redshift(user, passw, host, port, database)

In [13]:
# creamos la tabla en redshift, puede ser a traves de un query o con pandas

def create_table_query(conn, engine, schema, table_name):
    query = f'''
    CREATE TABLE IF NOT EXISTS {schema}.{table_name}(
    id VARCHAR(50) PRIMARY KEY,
    created_at TIMESTAMP,
    url VARCHAR(500),
    value VARCHAR(1000),
    fecha_insercion TIMESTAMP
    )
    '''
    conn.execute(query)
    return


create_table_query(conn, engine, schema, 'chuck_jokes2')

In [14]:
#Insert de un registro
get_joke().to_sql(
    name='chuck_jokes2',
    con=engine,
    schema=schema,
    if_exists='append',
    index=False,
    dtype={ 'id': sqlalchemy.types.VARCHAR(50),
            'created_at': sqlalchemy.types.TIMESTAMP,
            'url': sqlalchemy.types.VARCHAR(500),
            'value': sqlalchemy.types.VARCHAR(1000),
            'fecha_insercion': sqlalchemy.types.TIMESTAMP
            }
)

1

In [15]:
#Insertar un registro sin repetición de id

def insertar_sin_repeticion(conn, engine, schema, table_name):

    #Obtener registro de id de chistes
    query = f'SELECT id FROM {schema}.{table_name}'

    #Obtener ids de chistes en la tabla
    ids_en_tabla = pd.read_sql(query, conn)['id'].values

    #Nos aseguramos que el id no este en la tabla, si está, obtenemos otro chiste.
    while True:
        data = get_joke()
        joke_id = data['id'][0]

        if joke_id not in ids_en_tabla:
            break
        else:
            print('Chiste repetido, obteniendo otro chiste')
    
    data.to_sql(
            name='chuck_jokes2',
            con=engine,
            schema=schema,
            if_exists='append',
            index=False,
            dtype={ 'id': sqlalchemy.types.VARCHAR(50),
                    'created_at': sqlalchemy.types.TIMESTAMP,
                    'url': sqlalchemy.types.VARCHAR(500),
                    'value': sqlalchemy.types.VARCHAR(1000),
                    'fecha_insercion': sqlalchemy.types.TIMESTAMP
                    }
    )

    return

insertar_sin_repeticion(conn, engine, schema, 'chuck_jokes2')